In [9]:
import os
import shutil
from itertools import combinations
import numpy as np
import pandas as pd
from tqdm import tqdm

current_directory = os.getcwd()
print("Current working directory:", current_directory)
print("")

source_folder = '../../'
print("Source folder:", os.path.abspath(source_folder))

dst_folder = current_directory
print("Destination folder:", os.path.abspath(dst_folder))

Current working directory: /Users/ywu/InSync/2021_MPIE/2021-12_H Diffusion/Matlab Toolbox HEA/v6_A-B-C-D-E_Sputtering/CCA_CALPHAD_SSS_ML/v6_A-B-C-D-E_Sputtering_ML

Source folder: /Users/ywu/InSync/2021_MPIE/2021-12_H Diffusion/Matlab Toolbox HEA/v6_A-B-C-D-E_Sputtering
Destination folder: /Users/ywu/InSync/2021_MPIE/2021-12_H Diffusion/Matlab Toolbox HEA/v6_A-B-C-D-E_Sputtering/CCA_CALPHAD_SSS_ML/v6_A-B-C-D-E_Sputtering_ML


In [10]:
comb_D_E = list(combinations(
    ['Co', 'V', 'Mn', 'Mo', 'Cu', 'Nb', 'W', 'Ti', 'Al', 'Si', 'Ta'], 2))
print(comb_D_E[0])

('Co', 'V')


### Move the CALPHAD generated files to the new folder

1. in at%
2. in wt%


In [11]:
for comb in tqdm(comb_D_E):

    comb_A_B_C_D_E_temp = 'Fe_Cr_Ni_' + comb[0] + '_' + comb[1]
    print("")
    print(comb_A_B_C_D_E_temp)

    # Define the source directory -----------------------------------
    source_subfolder = 'v6_' + comb_A_B_C_D_E_temp + '_Sputtering'
    source_file = os.path.join(
        source_folder, source_subfolder, 'SSS_FCC_byCompo.xlsx')

    # print('Source file directory:', os.path.abspath(source_file))

    if os.path.isfile(source_file):
        print(comb_A_B_C_D_E_temp + ': File exists!')
    else:
        print(comb_A_B_C_D_E_temp + ': File does not exist.')
    # ----------------------------------------------------------------

    # read the source file and convert to at.% -----------------------
    df_raw = pd.read_excel(source_file)
    # display(df_raw.head())
    # copy the df_raw
    df_raw_at = df_raw.copy()

    df_raw_at['Fe'] = df_raw_at['Fe'] * 100
    df_raw_at['Cr'] = df_raw_at['Cr'] * 100
    df_raw_at['Ni'] = df_raw_at['Ni'] * 100
    df_raw_at[comb[0]] = df_raw_at[comb[0]] * 100
    df_raw_at[comb[1]] = df_raw_at[comb[1]] * 100

    # display(df_raw_at.head())
    # ----------------------------------------------------------------

    # read the source file and convert to wt.% -----------------------
    df_raw_wt = df_raw.copy()

    ele_A = 'Fe'
    ele_B = 'Cr'
    ele_C = 'Ni'
    ele_D = comb[0]
    ele_E = comb[1]

    ele_all = ['Fe', 'Cr', 'Ni', 'Co', 'V', 'Mn',
               'Mo', 'Cu', 'Nb', 'W', 'Ti', 'Al', 'Si', 'Ta']

    ele_indices = [ele_all.index(ele)
                   for ele in [ele_A, ele_B, ele_C, ele_D, ele_E]]
    # print(ele_indices)

    # Define the atomic weights of all elements
    atomic_wt_all = np.array([55.847, 51.996, 58.70, 58.9332, 50.9415, 54.9380, 95.94, 63.546, 92.9064, 183.85, 47.90,
                              26.98154, 28.0855, 180.9479])  # Source: https://www.angelo.edu/faculty/kboudrea/periodic/structure_mass.htm

    # Extract the atomic weights of the elements in A, B, C, D, and E
    atomic_wt_ABCDE = atomic_wt_all[ele_indices]

    # Calculate the total atomic weight of each composition
    total_atomic_wt = np.sum(df_raw_wt.loc[:, [
        ele_A, ele_B, ele_C, ele_D, ele_E]].values * atomic_wt_ABCDE, axis=1)
    # display(total_atomic_wt)

    # Calculate the weight percentages of each element for each composition
    df_raw_wt.loc[:, ele_A] = df_raw_wt.loc[:, ele_A] * \
        atomic_wt_ABCDE[0] / total_atomic_wt * 100
    df_raw_wt.loc[:, ele_B] = df_raw_wt.loc[:, ele_B] * \
        atomic_wt_ABCDE[1] / total_atomic_wt * 100
    df_raw_wt.loc[:, ele_C] = df_raw_wt.loc[:, ele_C] * \
        atomic_wt_ABCDE[2] / total_atomic_wt * 100
    df_raw_wt.loc[:, ele_D] = df_raw_wt.loc[:, ele_D] * \
        atomic_wt_ABCDE[3] / total_atomic_wt * 100
    df_raw_wt.loc[:, ele_E] = df_raw_wt.loc[:, ele_E] * \
        atomic_wt_ABCDE[4] / total_atomic_wt * 100

    # display(df_raw_wt.head())
    # display(df_raw_wt.tail())
    # ----------------------------------------------------------------

    # Define the destination directory and to_excel ------------------
    dst_subfolder = 'v6_A-B-C-D-E_Sputtering_ML_All'
    dst_file_at = os.path.join(
        dst_folder, dst_subfolder, 'v6_' + comb_A_B_C_D_E_temp + '_SSS_FCC_byCompo_at_pct.xlsx')
    dst_file_wt = os.path.join(
        dst_folder, dst_subfolder, 'v6_' + comb_A_B_C_D_E_temp + '_SSS_FCC_byCompo_wt_pct.xlsx')
    # print('Destination file directory:', os.path.abspath(dst_file_at))
    # print('Destination file directory:', os.path.abspath(dst_file_wt))
    df_raw_at.to_excel(dst_file_at, index=False)
    df_raw_wt.to_excel(dst_file_wt, index=False)
    print(comb_A_B_C_D_E_temp + ': File copied successfully!')
    # ----------------------------------------------------------------

    # # Copy the file
    # try:
    #     # Copy the file to the destination directory
    #     shutil.copyfile(source_file, dst_file_at)
    #     print(comb_A_B_C_D_E_temp + ': File copied successfully!')
    #     print("")
    # except Exception as e:
    #     print('Error copying file: ' + str(e))

  0%|          | 0/55 [00:00<?, ?it/s]

Fe_Cr_Ni_Co_V
Fe_Cr_Ni_Co_V: File exists!


  2%|▏         | 1/55 [00:08<07:13,  8.02s/it]

Fe_Cr_Ni_Co_V: File copied successfully!
Fe_Cr_Ni_Co_Mn
Fe_Cr_Ni_Co_Mn: File exists!


  4%|▎         | 2/55 [00:15<06:36,  7.49s/it]

...

Fe_Cr_Ni_Mo_Cu: File copied successfully!
Fe_Cr_Ni_Mo_Nb
Fe_Cr_Ni_Mo_Nb: File exists!


 53%|█████▎    | 29/55 [03:41<03:12,  7.41s/it]

Fe_Cr_Ni_Mo_Nb: File copied successfully!
Fe_Cr_Ni_Mo_W
Fe_Cr_Ni_Mo_W: File exists!


 53%|█████▎    | 29/55 [03:46<03:22,  7.81s/it]


KeyboardInterrupt: 